### Read SLHA, SModelS output and store the data in a pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels')
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.particlesLoader import getParticlesFromSLHA

pd.options.mode.chained_assignment = None #Disable copy warnings

/tmp/ipykernel_2881/1716768401.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp,os,sys


In [2]:
BSMlist = getParticlesFromSLHA('/home/yoxara/2MDM/2mdm_slha_files_all/run_01_zp_3612_dm_1412_gqv_06.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, zp, ghzp, ghzp~, sd, chi]


In [3]:
slhaFolder = '/home/yoxara/2MDM/2mdm_slha_files_all'
smodelsFolder = '/home/yoxara/2MDM/notebooks/SmodelS/2mdm_all_results/'

In [4]:
!runSModelS.py -p /home/yoxara/2MDM/notebooks/SmodelS/parameters_2mdm.ini -f /home/yoxara/2MDM/2mdm_slha_files_all -o /home/yoxara/2MDM/notebooks/SmodelS/2mdm_all_results/

INFO in databaseObj.loadBinaryFile() in 526: loading binary db file /home/yoxara/smodels/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 533: Loaded database from /home/yoxara/smodels/smodels-database/db3.pcl in 13.0 secs.
INFO in modelTester.loadDatabaseResults() in 498: Including non-validated results
INFO in modelTester.testPoints() in 358: Running SModelS for 1870 files with a single process. Messages will be redirected to smodels.log


In [5]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(smodelsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [6]:
print(len(data))

3740


In [7]:
#Convert data to flat DataFrame:
smodelsDF = json_normalize(data)

In [8]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 9000006])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['FRBLOCK'].items()])
    extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 9000006])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [9]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [10]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [11]:

print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 7480


In [12]:
#Save DataFrame to pickle file:
dataDF.to_pickle('/home/yoxara/2MDM/notebooks/SmodelS/2MDM_scanResults_all.pcl')

In [13]:
print(len(dataDF))

7480


In [21]:
from smodels.experiment.databaseObj import Database
path = "/home/yoxara/smodels/smodels-database" # Path to database
db = Database(path) # Load database and model
#analysis_list = ['all']%['CMS-PAS-EXO-20-008', 'ATLAS-EXOT-2018-48', 'ATLAS-EXOT-2019-03', 'CMS-EXO-19-012','CMS-EXO-20-004-eff']
#db.selectExpResults(analysisIDs=analysis_list, txnames='all')#, useNonValidated=True)#, dataTypes='upperLimit'
print(db)

Database version: 3.0.0-beta
-----------------------------
206 experimental results: 93 CMS, 113 ATLAS, 77 @ 8 TeV, 129 @ 13 TeV
1322 datasets, 7269 txnames.



In [30]:
print(db, db.getExpResults('ATLAS-EXOT-2019-03'))
print(db, db.getExpResults('CMS-PAS-EXO-20-008'))
print(db, db.getExpResults('ATLAS-EXOT-2018-48'))
print(db, db.getExpResults('ATLAS-EXOT-2019-03'))
print(db, db.getExpResults('CMS-SUS-20-004'))



Database version: 3.0.0-beta
-----------------------------
1 experimental results: 0 CMS, 1 ATLAS, 0 @ 8 TeV, 1 @ 13 TeV
1 datasets, 2 txnames.
Database version: 3.0.0-beta
-----------------------------
1 experimental results: 1 CMS, 0 ATLAS, 0 @ 8 TeV, 1 @ 13 TeV
1 datasets, 1 txnames.
Database version: 3.0.0-beta
-----------------------------
1 experimental results: 0 CMS, 1 ATLAS, 0 @ 8 TeV, 1 @ 13 TeV
1 datasets, 1 txnames.
Database version: 3.0.0-beta
-----------------------------
1 experimental results: 0 CMS, 1 ATLAS, 0 @ 8 TeV, 1 @ 13 TeV
1 datasets, 2 txnames.
Database version: 3.0.0-beta
-----------------------------
2 experimental results: 2 CMS, 0 ATLAS, 0 @ 8 TeV, 2 @ 13 TeV
23 datasets, 46 txnames.
 [<smodels.experiment.expResultObj.ExpResult object at 0x7f7093573fa0>, <smodels.experiment.expResultObj.ExpResult object at 0x7f7093571ae0>]
